In [3]:
import os
import image_processing
import cupy as cp

In [4]:
file_path = "data/lidar/Job1051007_34077_04_88.laz"

min_x_extent = 712160
max_x_extent = 712230
min_y_extent = 33100
max_y_extent = 33170

grid_gen = image_processing.GridGenerator(
    file_path, min_x_extent, max_x_extent, min_y_extent, max_y_extent
)

resolution = 0.05  # meters

pts_array = grid_gen.create_point_array()
grid_x, grid_y, grid_z = grid_gen.gen_grid(resolution, z=pts_array)

Directory to store grids already exists: generated_grids


ContainsArrayError: An array exists in store LocalStore('file://generated_grids/Job1051007_34077_04_88.laz_grid_x_0.05m.zarr') at path ''.

In [4]:
grid_z_gpu = cp.asarray(grid_z)

mapper = image_processing.DepthMapper(grid_z_gpu)

subfolder_path = 'data/CB_03_20220909212130_20220910054130'
labels_rects_zarr_folder = os.path.join(subfolder_path, "zarr", "labels_rects")

# Check if the labels_rects folder exists
if os.path.exists(labels_rects_zarr_folder):
    # print(f"Processing folder {subfolder}.")
    depth_map_zarr_save_dir = os.path.join(
        subfolder_path, "zarr", "depth_maps_updated"
    )
    pond_edge_elev_plot_dir = os.path.join(subfolder_path, 'plots', 'edge_histograms')
    mapper.process_depth_maps(labels_rects_zarr_folder, depth_map_zarr_save_dir, pond_edge_elev_plot_dir)

In [ ]:
if os.path.exists(depth_map_zarr_dir):
            for file_name in sorted(os.listdir(depth_map_zarr_dir)):
                if file_name.endswith("_ponding"):
                    timestamp = image_processing.image_utils.extract_timestamp(
                        file_name
                    )
                    date = pd.to_datetime(timestamp, utc=True)

                    orig_file_name = next(
                        (
                            f
                            for f in os.listdir(orig_image_rects_zarr_dir)
                            if image_processing.image_utils.extract_timestamp(f)
                            == timestamp
                        ),
                        None,
                    )

                    if orig_file_name is None:
                        print(
                            f"Warning: No matching original image found for {file_name}"
                        )
                        continue

                    orig_zarr_store_path = os.path.join(
                        orig_image_rects_zarr_dir, orig_file_name
                    )
                    orig_img_store = zarr.open(orig_zarr_store_path, mode="r")
                    orig_image = orig_img_store[:]  # Consider downsampling if necessary

                    zarr_store_path = os.path.join(depth_map_zarr_dir, file_name)
                    img_store = zarr.open(zarr_store_path, mode="r")
                    depth_map = img_store[
                        :
                    ]  # Again, consider loading only necessary slices

                    # print(f"Processing depth map: {file_name}")

                    fig, (ax1, ax3) = plt.subplots(
                        2, 1, figsize=(12, 12), gridspec_kw={"height_ratios": [1.5, 1]}
                    )

                    # Plot depth map
                    im = ax1.imshow(
                        orig_image, cmap="gray"
                    )  # Assuming orig_image is grayscale
                    im = ax1.imshow(
                        depth_map, cmap=cmocean.cm.deep, vmin=depth_min, vmax=depth_max
                    )

                    cbar = plt.colorbar(im, ax=ax1, label="Depth")
                    cbar.set_label("Depth (m)")
                    ax1.invert_yaxis()
                    ax1.set_xlabel("X (cm)")
                    ax1.set_ylabel("Y (cm)")
                    ax1.text(
                        0.05,
                        0.95,
                        f"Spatial Extent ($m^2$): {round((np.sum(~np.isnan(depth_map))) * 0.0025, 2)}",
                        transform=ax1.transAxes,
                        fontsize=12,
                        verticalalignment="top",
                        bbox=dict(facecolor="white", alpha=0.8, edgecolor="black"),
                    )

In [11]:
import zarr
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from pathlib import Path
import rioxarray
import xarray as xr
import numpy as np
from shapely.geometry import Polygon
from affine import Affine

In [25]:


def plot_flood_from_numpy(numpy_array, min_x, max_x, min_y, max_y, 
                          resolution_m=0.05, bbox_crs='EPSG:2264', output_folder='figures'):
    """
    Creates a georeferenced plot of a flood depth numpy array on a satellite basemap.

    Args:
        numpy_array (np.ndarray): 2D numpy array of flood depth data.
        min_x (float): The minimum x-coordinate of the bounding box.
        max_x (float): The maximum x-coordinate of the bounding box.
        min_y (float): The minimum y-coordinate of the bounding box.
        max_y (float): The maximum y-coordinate of the bounding box.
        resolution_m (float, optional): The resolution of the array in meters. Defaults to 0.05 (5cm).
        bbox_crs (str, optional): The CRS of the input coordinates. 
                                  'EPSG:32119' for NC State Plane (meters).
                                  'EPSG:2264' for NC State Plane (US survey feet).
        output_folder (str, optional): Folder to save the output image. Defaults to 'figures'.
    """
    
    # 1. Build the georeferenced DataArray in its native CRS
    transform = Affine(resolution_m, 0.0, min_x, 0.0, -resolution_m, max_y)
    da_hmax = xr.DataArray(
        data=numpy_array,
        dims=["y", "x"],
        name='flood_depth',
        attrs={'units': 'm'}
    )
    da_hmax.rio.write_crs(bbox_crs, inplace=True)
    da_hmax.rio.write_transform(transform, inplace=True)
    
    # 2. Create the plot axes
    fig, ax = plt.subplots(figsize=(10, 10))

    # --- NEW, ROBUST PLOTTING LOGIC ---

    # 3. Reproject the raster ONCE to the map's CRS (Web Mercator).
    # This becomes our single source of truth for all plot coordinates.
    da_hmax_mercator = da_hmax.rio.reproject(3857)

    # 4. Get the exact spatial bounds DIRECTLY from the reprojected raster.
    minx, miny, maxx, maxy = da_hmax_mercator.rio.bounds()

    # 5. Set the axis limits to perfectly match the raster's extent.
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # 6. Plot the reprojected raster. It will now fit the frame perfectly.
    da_hmax_mercator.plot(
        ax=ax,
        cmap='Blues',
        alpha=0.7,
        add_colorbar=True,
        vmin=0.01,
        cbar_kwargs={
            'label': 'Flood Depth (m)',
            'shrink': 0.6,
            'aspect': 30
        }
    )

    # 7. Create and plot an outline that matches the raster's true bounds.
    outline_gdf = gpd.GeoDataFrame(geometry=[box(minx, miny, maxx, maxy)], crs="EPSG:3857")
    outline_gdf.plot(ax=ax, edgecolor='cyan', facecolor='none', linewidth=2)
    
    # 8. Add the basemap. It will use the limits we set, which are now guaranteed to match the data.
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    
    # 9. Clean up and save
    png_path = Path(output_folder) / "flood_map_final.png"
    png_path.parent.mkdir(parents=True, exist_ok=True)
    
    ax.set_title('')
    ax.set_axis_off()
    plt.tight_layout()
    
    plt.savefig(png_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"Plot with flood data and basemap saved to {png_path}.")

In [6]:
import zarr
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from pathlib import Path
import rioxarray
import xarray as xr
import numpy as np
from affine import Affine

# Imports needed for the manual colorbar
from matplotlib import cm
from matplotlib.colors import Normalize

def plot_flood_from_numpy_imshow(numpy_array, min_x, max_x, min_y, max_y,
                                 resolution_m=0.05, bbox_crs='EPSG:32119', output_folder='figures'):
    """
    Final attempt using low-level ax.imshow to bypass potential xarray.plot conflicts.
    """
    # 1. Build and georeference the DataArray in its native CRS
    transform = Affine(resolution_m, 0.0, min_x, 0.0, -resolution_m, max_y)
    da_hmax = xr.DataArray(
        data=numpy_array.astype(float),
        dims=["y", "x"],
        name='flood_depth'
    )
    da_hmax.rio.write_crs(bbox_crs, inplace=True)
    da_hmax.rio.write_transform(transform, inplace=True)
    # Explicitly set nodata to np.nan to match your data format
    da_hmax.rio.write_nodata(np.nan, inplace=True)

    # 2. Reproject the raster to the map's CRS (Web Mercator).
    da_hmax_mercator = da_hmax.rio.reproject(3857)

    # 3. Create the plot axes
    fig, ax = plt.subplots(figsize=(10, 10))

    # --- NEW RENDERING LOGIC ---

    # 4. Get the exact spatial bounds and data from the reprojected raster.
    minx, miny, maxx, maxy = da_hmax_mercator.rio.bounds()
    data_to_plot = da_hmax_mercator.to_numpy()
    
    # By default, imshow doesn't plot nan values, so they will be transparent.

    # 5. Set the axis limits to perfectly match the raster's extent.
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # 6. Use ax.imshow() to plot the data directly.
    # extent is (left, right, bottom, top)
    im = ax.imshow(
        data_to_plot,
        extent=(minx, maxx, miny, maxy),
        cmap='Blues',
        alpha=0.7,
        interpolation='none' # Use 'none' for raw pixel data
    )

    # 7. Add the basemap.
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    # 8. Manually create a colorbar for the imshow plot
    # Create a normalizer for the colormap
    # We use nanmin and nanmax to ignore the nan values when setting the color limits
    norm = Normalize(vmin=np.nanmin(data_to_plot), vmax=np.nanmax(data_to_plot))
    # Create the colorbar
    cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap='Blues'), ax=ax, shrink=0.6, aspect=30)
    cbar.set_label('Flood Depth (m)')
    
    # 9. Clean up and save
    png_path = Path(output_folder) / "flood_map_final_imshow.png"
    png_path.parent.mkdir(parents=True, exist_ok=True)

    ax.set_title('')
    ax.set_axis_off()
    plt.tight_layout()

    plt.savefig(png_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"Plot created with imshow saved to {png_path}.")

In [8]:
zarr_store_path = 'data/CB_03_20220909212130_20220910054130/zarr/depth_maps_updated/CAM_CB_03_20220910000631_predseg_labels_rectified_depth_map_90_perc'

img_store = zarr.open(zarr_store_path, mode="r")
depth_map = img_store[:]

plot_flood_from_numpy_imshow(depth_map, min_x_extent, max_x_extent, min_y_extent, max_y_extent, bbox_crs='EPSG:2264', output_folder='figures')

Plot created with imshow saved to figures/flood_map_final_imshow.png.
